# Neural Networks and GLoVe modeling

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

Using TensorFlow backend.


## Read in dataframe

In [3]:
# df1 from model_data.csv has tweets that have punctuation and are not lemmatized.
# df1 = pd.read_csv('model_data.csv')

In [4]:
df = pd.read_csv('CLEAN_EDIT_model_data.csv')

In [5]:
df.drop(columns = ['Unnamed: 0'], inplace= True)

In [6]:
df.head(1)

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# set the tweet column to a string to account of data type errors
df.tweet = df.tweet.astype(str)

In [8]:
# to save the clean dataframe
# df.to_csv(r'/Users/sproul/Desktop/ds-projects/project_mod4_AAR/maybe_final_data.csv')

### Check the class balance of our target classes

In [9]:
target = df.signal_x

In [10]:
df["signal_x"].value_counts()

stay    882
up      679
down    641
Name: signal_x, dtype: int64

In [11]:
# Make a histogram to show the distribution of target classes

In [12]:
df["signal_x"].value_counts(normalize=True)

stay    0.400545
up      0.308356
down    0.291099
Name: signal_x, dtype: float64

### Set target to the daily move in stock price (or signal_x)

In [13]:
target = df.signal_x

In [14]:
target.head()

0    stay
1    stay
2    stay
3    stay
4    stay
Name: signal_x, dtype: object

### Set the target to an array of dummies for our three classes (stay, up, down)

We use this y (the target) for all of our models

In [15]:
y = pd.get_dummies(target).values

In [16]:
print(type(y))
y[:4]

<type 'numpy.ndarray'>


array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

# Initial ANN Model

### The initial data for our first model is just the text of the tweets

In [17]:
data = df['tweet'].map(word_tokenize).values

In [18]:
print(type(data))
data

<type 'numpy.ndarray'>


array([list(['assuming', 'acceleration', 'of', 'to', 'but', 'in', 'a', 'comfortable', 'direction', 'will', 'feel', 'like', 'a', 'mild', 'to']),
       list(['is', 'capable', 'of', 'transporting', 'satellite', 'to', 'orbit', 'crew', 'and', 'cargo', 'to', 'the', 'and', 'mission', 'to', 'the', 'moon', 'an']),
       list(['yup']), ...,
       list(['these', 'article', 'in', 'space', 'news', 'describe', 'why', 'v', 't', 'and', 't', 'w']),
       list(['wa', 'by', 'a', 'saying', 'rocket', 'ha', 'no', 'chance', 'just', 'said', 'the', 'franco', 'german', 'ha', 'no', 'chance', 'so', 'go', 'with']),
       list(['just', 'returned', 'from', 'a', 'trip', 'to', 'and', 'where', 'i', 'met', 'with', 'many', 'interesting', 'people', 'i', 'really', 'like'])],
      dtype=object)

Tokenize the words with keras preprocessing

In [19]:
tokenizer = text.Tokenizer(num_words=15000)
tokenizer.fit_on_texts(list(df.tweet))
list_tokenized_tweets = tokenizer.texts_to_sequences(df.tweet)
X_t = sequence.pad_sequences(list_tokenized_tweets) #, maxlen=2709)

In [20]:
print('this is a list of arrays of numbers for each tweet')
print(type(list_tokenized_tweets))
list_tokenized_tweets[:3]

this is a list of arrays of numbers for each tweet
<type 'list'>


[[979, 532, 4, 2, 20, 6, 3, 1784, 1258, 12, 386, 36, 3, 1785, 2],
 [5, 805, 4, 1259, 184, 2, 135, 416, 8, 457, 2, 1, 8, 151, 2, 1, 285, 46],
 [387]]

In [21]:
print(type(X_t))
print(X_t.shape)
X_t[:3]

<type 'numpy.ndarray'>
(2202, 28)


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  979,  532,    4,    2,   20,    6,    3, 1784, 1258,
          12,  386,   36,    3, 1785,    2],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    5,
         805,    4, 1259,  184,    2,  135,  416,    8,  457,    2,    1,
           8,  151,    2,    1,  285,   46],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  387]], dtype=int32)

### Split the data into test and train data

In [22]:
X_train_tweet, X_test_tweet, y_train_tweet, y_test_tweet = train_test_split(X_t, y, test_size = 0.2, random_state = 0)


In [23]:
embedding_size = 28 # another test is 128
input_ = Input(shape=(28,)) # another test is 100
x = Embedding(10000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 3 different possible classes, so we use 3 neurons in our output layer
x = Dense(3, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 28, 28)            280000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 28, 25)            5400      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 25)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [26]:
model.fit(X_train_tweet, y_train_tweet, epochs=20, batch_size=32, validation_split=0.1)


Train on 1584 samples, validate on 177 samples
Epoch 1/20
1584/1584 [==============================] - 8s 5ms/step - loss: 1.0961 - acc: 0.3731 - val_loss: 1.0950 - val_acc: 0.3729
Epoch 2/20
1584/1584 [==============================] - 6s 4ms/step - loss: 1.0901 - acc: 0.4078 - val_loss: 1.0952 - val_acc: 0.3729
Epoch 3/20
1584/1584 [==============================] - 6s 4ms/step - loss: 1.0871 - acc: 0.4078 - val_loss: 1.0967 - val_acc: 0.3729
Epoch 4/20
1584/1584 [==============================] - 5s 3ms/step - loss: 1.0774 - acc: 0.4091 - val_loss: 1.0942 - val_acc: 0.3729
Epoch 5/20
1584/1584 [==============================] - 6s 4ms/step - loss: 1.0399 - acc: 0.4602 - val_loss: 1.0963 - val_acc: 0.3672
Epoch 6/20
1584/1584 [==============================] - 6s 4ms/step - loss: 0.9127 - acc: 0.5612 - val_loss: 1.1358 - val_acc: 0.3842
Epoch 7/20
1584/1584 [==============================] - 6s 4ms/step - loss: 0.7966 - acc: 0.6199 - val_loss: 1.4368 - val_acc: 0.3333
Epoch 8/20
1584

In [27]:
score_tweet, acc_tweet = model.evaluate(X_test_tweet, y_test_tweet, verbose = 2, batch_size = 32)
print("score: %.2f" % (score_tweet))
print("acc: %.2f" % (acc_tweet))

score: 4.00
acc: 0.39


This model is the base line Neural Network, with an accuracy of 43%

---

# New model experiments with Random Forest, Support Vector Machine, and Logistic Regression 

Next, we are testing other models, to get a general baseline before optimizing the Neural Networks

#### GloVe

In [43]:
total_vocabulary = set(word for tweet in data for word in tweet)
len(total_vocabulary)

3235

In [44]:
# read in that text file

In [45]:
glove = {}
with open('glove.6B.100d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [46]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [47]:
rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [48]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [49]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/Users/roberthillery/anaconda3/envs/venv/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/roberthillery/anaconda3/envs/venv/lib/python2.7/site-pac

In [50]:
scores

[('Random Forest', 0.3696658967167134),
 ('Support Vector Machine', 0.40009074410163337),
 ('Logistic Regression', 0.36239894406863554)]

Random Forest cross validation score 0.3696658967167134

Support Vector Machine cross validation score 0.40009074410163337


Logistic Regression cross validation score 0.36239894406863554

# ANN Model

In [28]:

df.head()

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Check the tweet data going into the model
X_t[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  979,  532,    4,    2,   20,    6,    3, 1784, 1258,
          12,  386,   36,    3, 1785,    2],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    5,
         805,    4, 1259,  184,    2,  135,  416,    8,  457,    2,    1,
           8,  151,    2,    1,  285,   46],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  387]], dtype=int32)

In [30]:
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(X_t, y, test_size = 0.2, random_state = 0)


## Build the model

In [31]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# ann = classifier.fit(train_final, label_train_final, batch_size = 10, epochs = 100, validation_data=(val, label_val))
ann = classifier.fit(X_train_ann, y_train_ann, batch_size = 10, epochs = 100, validation_split=0.1)

Train on 1584 samples, validate on 177 samples
Epoch 1/100
1584/1584 [==============================] - 1s 933us/step - loss: 0.6386 - acc: 0.6593 - val_loss: 0.6348 - val_acc: 0.6685
Epoch 2/100
1584/1584 [==============================] - 1s 325us/step - loss: 0.6303 - acc: 0.6654 - val_loss: 0.6371 - val_acc: 0.6591
Epoch 3/100
1584/1584 [==============================] - 1s 324us/step - loss: 0.6241 - acc: 0.6679 - val_loss: 0.6397 - val_acc: 0.6497
Epoch 4/100
1584/1584 [==============================] - 1s 326us/step - loss: 0.6215 - acc: 0.6698 - val_loss: 0.6401 - val_acc: 0.6648
Epoch 5/100
1584/1584 [==============================] - 1s 317us/step - loss: 0.6174 - acc: 0.6738 - val_loss: 0.6629 - val_acc: 0.6441
Epoch 6/100
1584/1584 [==============================] - 1s 329us/step - loss: 0.6128 - acc: 0.6772 - val_loss: 0.6551 - val_acc: 0.6535
Epoch 7/100
1584/1584 [==============================] - 1s 320us/step - loss: 0.6059 - acc: 0.6787 - val_loss: 0.6392 - val_acc: 0

1584/1584 [==============================] - 0s 266us/step - loss: 0.4937 - acc: 0.7445 - val_loss: 0.9492 - val_acc: 0.6667
Epoch 61/100
1584/1584 [==============================] - 0s 283us/step - loss: 0.5103 - acc: 0.7401 - val_loss: 0.9111 - val_acc: 0.6610
Epoch 62/100
1584/1584 [==============================] - 0s 262us/step - loss: 0.4954 - acc: 0.7449 - val_loss: 0.9176 - val_acc: 0.6629
Epoch 63/100
1584/1584 [==============================] - 0s 278us/step - loss: 0.4886 - acc: 0.7475 - val_loss: 0.9461 - val_acc: 0.6648
Epoch 64/100
1584/1584 [==============================] - 0s 274us/step - loss: 0.4889 - acc: 0.7504 - val_loss: 0.9204 - val_acc: 0.6648
Epoch 65/100
1584/1584 [==============================] - 0s 275us/step - loss: 0.4925 - acc: 0.7498 - val_loss: 0.8993 - val_acc: 0.6798
Epoch 66/100
1584/1584 [==============================] - 0s 273us/step - loss: 0.4971 - acc: 0.7496 - val_loss: 1.0424 - val_acc: 0.6535
Epoch 67/100
1584/1584 [=======================

In [32]:
results_train_ann = classifier.evaluate(X_train_ann, y_train_ann)

1761/1761 [==============================] - 0s 32us/step


In [33]:
results_test_ann = classifier.evaluate(X_test_ann, y_test_ann)

441/441 [==============================] - 0s 38us/step


In [34]:
print(results_train_ann)
print(results_test_ann)

[0.5391556257757526, 0.7482490954854013]
[1.2146780242184663, 0.6243386186845178]


In [35]:
score_ann, acc_ann = classifier.evaluate(X_test_ann, y_test_ann, verbose = 2, batch_size = 32)
print("score for ann model: %.2f" % (score_ann))
print("acc for ann model: %.2f" % (acc_ann))

score for ann model: 1.21
acc for ann model: 0.62



The neural network for tweets has a 62% accuracy rate.

# ANN for non tweet features

Make the dataframe for not tweet features, which include the tf-idf of the tweets.

In [36]:
df_no_tweet = df.drop(['signal_x', 'tweet'], axis=1)

In [37]:
df_no_tweet.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,ability,able,aboard,abort,aborted,about,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,137,105,1,0.366667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,64,113,1,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,137,6,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,137,7,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,137,96,1,0.650000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# these are the features for the no_tweet ANN model
X = df_no_tweet.as_matrix()

/Users/roberthillery/anaconda3/envs/venv/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [39]:
X[:3]

array([[137., 105.,   1., ...,   0.,   0.,   0.],
       [ 64., 113.,   1., ...,   0.,   0.,   0.],
       [137.,   6.,   0., ...,   0.,   0.,   0.]])

In [40]:
# this is check the target for the Neural Network
print(type(y))

<type 'numpy.ndarray'>


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [42]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1)

Train on 1584 samples, validate on 177 samples
Epoch 1/100
1584/1584 [==============================] - 3s 2ms/step - loss: 0.6326 - acc: 0.6667 - val_loss: 0.6386 - val_acc: 0.6667
Epoch 2/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6304 - acc: 0.6667 - val_loss: 0.6381 - val_acc: 0.6667
Epoch 3/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6312 - acc: 0.6667 - val_loss: 0.6373 - val_acc: 0.6667
Epoch 4/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6299 - acc: 0.6667 - val_loss: 0.6384 - val_acc: 0.6667
Epoch 5/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6297 - acc: 0.6667 - val_loss: 0.6385 - val_acc: 0.6667
Epoch 6/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6297 - acc: 0.6660 - val_loss: 0.6370 - val_acc: 0.6667
Epoch 7/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.6289 - acc: 0.6667 - val_loss: 0.6398 - val_acc: 0.6667
Epoch 8/

1584/1584 [==============================] - 2s 1ms/step - loss: 0.0803 - acc: 0.9657 - val_loss: 1.7735 - val_acc: 0.5518
Epoch 61/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0650 - acc: 0.9701 - val_loss: 2.0262 - val_acc: 0.5800
Epoch 62/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0726 - acc: 0.9697 - val_loss: 1.7778 - val_acc: 0.5593
Epoch 63/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0710 - acc: 0.9672 - val_loss: 1.8701 - val_acc: 0.5593
Epoch 64/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0621 - acc: 0.9743 - val_loss: 1.7668 - val_acc: 0.5499
Epoch 65/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0575 - acc: 0.9714 - val_loss: 1.8917 - val_acc: 0.5537
Epoch 66/100
1584/1584 [==============================] - 2s 1ms/step - loss: 0.0762 - acc: 0.9680 - val_loss: 2.0094 - val_acc: 0.5650
Epoch 67/100
1584/1584 [==============================] - 2s 

In [ ]:
score_no_tweet, acc_no_tweet = classifier.evaluate(X_test, y_test, verbose = 2, batch_size = 32)
print("score for no tweet model: %.2f" % (score_no_tweet))
print("acc for no tweet model: %.2f" % (acc_no_tweet))

The neural network for the non-tweet data has a 59% accuracy rate.

# Visualize the models

In [ ]:
# import matplotlib.pyplot as plt
# plt.clf()

# loss_values = model_val_dict['loss']
# val_loss_values = model_val_dict['val_loss']

# epochs = range(1, len(loss_values) + 1)
# plt.plot(epochs, loss_values, 'g', label='Training loss')
# plt.plot(epochs, val_loss_values, 'blue', label='Validation loss')

# plt.title('Training & validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()